'pip' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import os
import re
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from pdf2image import convert_from_path
from pytesseract import image_to_string

# Ensure Tesseract is installed and set the correct path if using Windows
if os.name == 'nt':  # Windows OS
    os.environ['TESSDATA_PREFIX'] = r"C:\Program Files\Tesseract-OCR\tessdata"
    tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    from pytesseract import pytesseract
    pytesseract.tesseract_cmd = tesseract_cmd

# Open file dialog to select a PDF file
root = tk.Tk()
root.withdraw()  # Hide the root window
pdf_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])

if not pdf_path:
    print("No file selected. Exiting...")
    exit()

print(f"Selected file: {pdf_path}")

# Convert PDF to images
pages = convert_from_path(pdf_path)
print(f"Number of pages converted: {len(pages)}")

# Extract text from images using OCR
ocr_text = ""
for i, page in enumerate(pages):
    page_text = image_to_string(page)
    ocr_text += f"--- Page {i+1} ---\n" + page_text + "\n"
    print(f"OCR completed for page {i+1}")

# Debugging step: print a preview of extracted text
print("\n===== OCR Extracted Text Preview (First 2000 characters) =====")
print(ocr_text[:2000])

# Function to parse extracted text
def parse_candidates(ocr_text):
    candidates = []
    pattern = re.compile(
        r"(?P<name>[A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\s-\s\d+°\n"
        r"(?P<title>.*?)\n\n"
        r"(?P<location>.*?)(?:\s-\s(?P<industry>.*?))\n\n"
        r"(?P<company_line>.*?)\n?\n"  # Capture the whole line containing company info
    )
    for match in pattern.finditer(ocr_text):
        candidate = match.groupdict()
        company_line = candidate.get('company_line', '')  # Get the company line

        # Extract the company name from the company line
        company_match = re.search(r"(?:presso|for|at)\s(.*?)(?:\s\d{4}|$)", company_line)
        if company_match:
            candidate["company"] = company_match.group(1).strip()  # Extract and clean company
        else:
            candidate["company"] = "Not Available"  # If no match, assign "Not Available"
        
        candidates.append(candidate)
        del candidate['company_line']  # Remove 'company_line' as it's not needed in the final output
    return candidates

# Parse extracted text
parsed_data = parse_candidates(ocr_text)
print(f"Parsed {len(parsed_data)} candidates")

# Save extracted data to Excel
def save_to_excel(data, output_file):
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)
    print(f"Data saved to {output_file}")

output_file = "candidates.xlsx"
save_to_excel(parsed_data, output_file)

print(f"Excel file saved successfully: {output_file}")


Selected file: C:/Users/sfarisse/Downloads/general2-5.pdf
Number of pages converted: 15
OCR completed for page 1
OCR completed for page 2
OCR completed for page 3
OCR completed for page 4
OCR completed for page 5
OCR completed for page 6
OCR completed for page 7
OCR completed for page 8
OCR completed for page 9
OCR completed for page 10
OCR completed for page 11
OCR completed for page 12
OCR completed for page 13
OCR completed for page 14
OCR completed for page 15

===== OCR Extracted Text Preview (First 2000 characters) =====
--- Page 1 ---
ES Recruiter Q @ a @ ®) =>

Spotlight a

110K+ 30K+ 21K+

Disponibili a lavorare Talenti attivi Candidati riscoperti

_J RISULTATI:280K+ Mostra dettagli ricerca ~
« 101-125 >

Marco Parente - 3°
5 Executive Manager
Anzio, Latium, Italy: Servizi IT e consulenza IT

Esperienza Security Advisor presso NTT DATA Italia - 2077 — Presente
Senior Security Consultant presso Freelance for Omnitech s.r.|. «2072 - 2017
Technical Leader presso SiliconDev Securi

In [ ]:
import requests
import pandas as pd

# 🔹 Load Excel file and clean column names
df = pd.read_excel("candidates_with_emails.xlsx")
df.columns = df.columns.str.strip()  # Remove extra spaces

# 🔹 Check if required columns exist
required_columns = ["company", "industry"]  # Expected column names
df.columns = df.columns.str.lower()  # Convert all columns to lowercase

if "company" not in df.columns or "settore" not in df.columns:
    print("❌ ERROR: The required columns 'Company' and 'Settore' were not found in the Excel file.")
    print("Available columns:", df.columns.tolist())
    exit()

# 🔹 Function to get company location using OpenStreetMap (Free API)
def get_location(company, industry):
    query = f"{company}, {industry}"
    url = f"https://nominatim.openstreetmap.org/search?format=json&q={query}"

    response = requests.get(url)
    data = response.json()

    if data:
        return {
            "Company": company,
            "Industry": industry,
            "Address": data[0]["display_name"],
            "Latitude": data[0]["lat"],
            "Longitude": data[0]["lon"],
            "OpenStreetMap Link": f"https://www.openstreetmap.org/?mlat={data[0]['lat']}&mlon={data[0]['lon']}"
        }
    else:
        return {
            "Company": company,
            "Industry": industry,
            "Address": "Not Found",
            "Latitude": "N/A",
            "Longitude": "N/A",
            "OpenStreetMap Link": "N/A"
        }

# 🔹 Store results in a list
locations = []

# 🔹 Search for each company in the Excel file
for _, row in df.iterrows():
    company = str(row["company"]).strip()
    industry = str(row["industry"]).strip()
    
    if company and industry:
        locations.append(get_location(company, industry))

# 🔹 Save results to a new Excel file
output_file = "company_locations.xlsx"
df_locations = pd.DataFrame(locations)
df_locations.to_excel(output_file, index=False)

print(f"✅ Locations saved successfully to {output_file}!")




❌ ERROR: The required columns 'Company' and 'Settore' were not found in the Excel file.
Available columns: ['name', 'title', 'location', 'industry', 'company', 'email', 'chatgpt insights', 'generated email']


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

: 